### Starbucks Capstone Challenge
# Feature Engineering

This notebook aims to generate features by preparing the data to correspond to the problem stated and feed the neural networks.  
The transcription records must be structured and labeled as an appropriate offer or not.

Feature Engineering is the process of determining which features might be useful in training a model, and then converting raw data from log files and other sources into said features.

In [1]:
## Import all the necessary libraries
import os

import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset

In [2]:
## Global definitions

data_dir = 'data'

pd.set_option('display.precision', 2)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

portfolio_data_path = os.path.join(data_dir, 'portfolio.json')
profile_data_path = os.path.join(data_dir, 'profile.json')
transcript_data_path = os.path.join(data_dir, 'transcript.json')

In [3]:
## global functions

def load_dataframe(data_path):
    """Create a dataframe from a json file"""
    return pd.read_json(data_path, orient='records', lines=True)

# Portfolio dataset
Dataset containing information about the offers which can be sent to customers.

## Overview

In [4]:
## Load dataset
portfolio_df = load_dataframe(portfolio_data_path)
display(portfolio_df.head())

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


## Transformation
As we can see **offer type** is a categorical feature that could be mapped as one hot encoding.  

**Channels** are categorical feature as well, but values can assume more than one category. Its values may be converted into individual features.  

In [5]:
## Set id as index
portfolio_df.set_index(keys='id', verify_integrity=True, inplace=True)

## Make offer_type one hot encoded
portfolio_df = portfolio_df.join(
    pd.get_dummies(portfolio_df.pop('offer_type')))

## Transform channels in distinct features
channels_df = pd.DataFrame(portfolio_df.pop('channels'))
channels_df = channels_df.explode('channels')
channels_df = channels_df.assign(value=lambda x: 1)
channels_df = channels_df.pivot(columns='channels', values='value')
channels_df.fillna(value=0, inplace=True)
portfolio_df = portfolio_df.join(channels_df)
channels_df = None

## print the result
display(portfolio_df)

,reward,difficulty,duration,bogo,discount,informational,email,mobile,social,web
id,,,,,,,,,,
ae264e3637204a6fb9bb56bc8210ddfd,10,10,7,1,0,0,1.0,1.0,1.0,0.0
4d5c57ea9a6940dd891ad53e9dbe8da0,10,10,5,1,0,0,1.0,1.0,1.0,1.0
3f207df678b143eea3cee63160fa8bed,0,0,4,0,0,1,1.0,1.0,0.0,1.0
9b98b8c7a33c4b65b9aebfe6a799e6d9,5,5,7,1,0,0,1.0,1.0,0.0,1.0
0b1e1539f2cc45b7b9fa7c272da2e1d7,5,20,10,0,1,0,1.0,0.0,0.0,1.0
2298d6c36e964ae4a3e7e9706d1fb8c2,3,7,7,0,1,0,1.0,1.0,1.0,1.0
fafdcd668e3743c1bb461111dcafc2a4,2,10,10,0,1,0,1.0,1.0,1.0,1.0
5a8bc65990b245e5a138643cd4eb9837,0,0,3,0,0,1,1.0,1.0,1.0,0.0
f19421c1d4aa40978ebb69ca19b0e20d,5,5,5,1,0,0,1.0,1.0,1.0,1.0


## Analysis

In [6]:
## Filter out email column, since it is not an informative feature
# once any offer uses this channel.
portfolio_df.drop(columns='email', inplace=True)

display(pd.DataFrame(portfolio_df.describe())
        .style.set_caption('Dataset description'))

display(portfolio_df.corr().abs()
        .style.set_caption('Pairwise correlation'))

,reward,difficulty,duration,bogo,discount,informational,mobile,social,web
count,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00
mean,4.20,7.70,6.50,0.40,0.40,0.20,0.90,0.60,0.80
std,3.58,5.83,2.32,0.52,0.52,0.42,0.32,0.52,0.42
min,0.00,0.00,3.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,2.00,5.00,5.00,0.00,0.00,0.00,1.00,0.00,1.00
50%,4.00,8.50,7.00,0.00,0.00,0.00,1.00,1.00,1.00
75%,5.00,10.00,7.00,1.00,1.00,0.00,1.00,1.00,1.00
max,10.00,20.00,10.00,1.00,1.00,1.00,1.00,1.00,1.00


,reward,difficulty,duration,bogo,discount,informational,mobile,social,web
reward,1.00,0.47,0.16,0.79,0.29,0.62,0.08,0.29,0.12
difficulty,0.47,1.00,0.81,0.03,0.60,0.70,0.74,0.15,0.24
duration,0.16,0.81,1.00,0.19,0.74,0.68,0.53,0.19,0.34
bogo,0.79,0.03,0.19,1.00,0.67,0.41,0.27,0.25,0.10
discount,0.29,0.60,0.74,0.67,1.00,0.41,0.41,0.17,0.41
informational,0.62,0.70,0.68,0.41,0.41,1.00,0.17,0.10,0.37
mobile,0.08,0.74,0.53,0.27,0.41,0.17,1.00,0.41,0.17
social,0.29,0.15,0.19,0.25,0.17,0.10,0.41,1.00,0.41
web,0.12,0.24,0.34,0.10,0.41,0.37,0.17,0.41,1.00


## Resulting dataset

In [7]:
display(portfolio_df)

,reward,difficulty,duration,bogo,discount,informational,mobile,social,web
id,,,,,,,,,
ae264e3637204a6fb9bb56bc8210ddfd,10,10,7,1,0,0,1.0,1.0,0.0
4d5c57ea9a6940dd891ad53e9dbe8da0,10,10,5,1,0,0,1.0,1.0,1.0
3f207df678b143eea3cee63160fa8bed,0,0,4,0,0,1,1.0,0.0,1.0
9b98b8c7a33c4b65b9aebfe6a799e6d9,5,5,7,1,0,0,1.0,0.0,1.0
0b1e1539f2cc45b7b9fa7c272da2e1d7,5,20,10,0,1,0,0.0,0.0,1.0
2298d6c36e964ae4a3e7e9706d1fb8c2,3,7,7,0,1,0,1.0,1.0,1.0
fafdcd668e3743c1bb461111dcafc2a4,2,10,10,0,1,0,1.0,1.0,1.0
5a8bc65990b245e5a138643cd4eb9837,0,0,3,0,0,1,1.0,1.0,0.0
f19421c1d4aa40978ebb69ca19b0e20d,5,5,5,1,0,0,1.0,1.0,1.0


# Profile data set
Dataset containing demographic data for each one of the reward program users.

## Overview

In [8]:
profile_df = load_dataframe(profile_data_path)
display(profile_df.head())

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


## Transformation

Altough the users with missing data could be filtered out of the data set, they might have some peculiar behaviour which would be interesting to be analyzed.  
Thus, my approach with those record is:  
* maintain those records in the data set
* mark those users as belonging to a particular class, by creating a new feature for that
* since gender is a discrete feature, create another gender category
* as age and income are continuous features, fill them with the respective mean values

The feature *became_member_on* is a date represented as a string, so that the values is difficulty to compare due to the lack of linearity.  
Consider the dates 20171230, 20171231, and 20180101. Altough the distances between them are the same (1 day), if they are considered as numbers the distance becames 1 and 8870, respectively.  
Thus, this feature should be transformed to the type long. 

In [9]:
# create new feature to indicate missing values
missing_data = profile_df.gender.isna()
profile_df = profile_df.assign(missing_data=missing_data.astype(int))

# fill gender if a new 'U' (unknown) category
profile_df.gender.mask(missing_data, 'U', inplace=True)

# fill age and income with its respective mean values

profile_df[missing_data].age.mean()

profile_df.age.mask(missing_data, profile_df[~missing_data].age.mean(), inplace=True)
profile_df.income.mask(missing_data, profile_df.income.mean(), inplace=True)

# Make gender one-hot encoded
profile_df = profile_df.join(
    pd.get_dummies(profile_df.pop('gender')))

# Set id as index
profile_df.set_index(keys='id', verify_integrity=True, inplace=True)

## Convert data to datetime format
profile_df.became_member_on = pd.to_datetime(profile_df.became_member_on, format='%Y%m%d').astype(np.long)

## Convert values to lower numbers
profile_df.became_member_on /= 1e17
profile_df.income /= 1e4
profile_df.age /= 10

In [10]:
display(pd.DataFrame(profile_df.describe()).round(2).style \
        .set_caption('Dataset description'))

display(profile_df.corr().abs().style \
        .set_caption('Pairwise correlation'))

,age,became_member_on,income,missing_data,F,M,O,U
count,17000.00,17000.00,17000.00,17000.00,17000.00,17000.00,17000.00,17000.00
mean,5.44,14.88,6.54,0.13,0.36,0.50,0.01,0.13
std,1.62,0.36,2.02,0.33,0.48,0.50,0.11,0.33
min,1.80,13.75,3.00,0.00,0.00,0.00,0.00,0.00
25%,4.50,14.64,5.10,0.00,0.00,0.00,0.00,0.00
50%,5.44,15.02,6.54,0.00,0.00,0.00,0.00,0.00
75%,6.50,15.15,7.60,0.00,1.00,1.00,0.00,0.00
max,10.10,15.33,12.00,1.00,1.00,1.00,1.00,1.00


,age,became_member_on,income,missing_data,F,M,O,U
age,1.00,0.01,0.31,0.00,0.15,0.14,0.00,0.00
became_member_on,0.01,1.00,0.02,0.03,0.01,0.03,0.01,0.03
income,0.31,0.02,1.00,0.00,0.22,0.21,0.01,0.00
missing_data,0.00,0.03,0.00,1.00,0.29,0.38,0.04,1.00
F,0.15,0.01,0.22,0.29,1.00,0.75,0.08,0.29
M,0.14,0.03,0.21,0.38,0.75,1.00,0.11,0.38
O,0.00,0.01,0.01,0.04,0.08,0.11,1.00,0.04
U,0.00,0.03,0.00,1.00,0.29,0.38,0.04,1.00


## Resulting dataset

In [11]:
display(profile_df.head(10))

,age,became_member_on,income,missing_data,F,M,O,U
id,,,,,,,,
68be06ca386d4c31939f3a4f0e3dd783,5.44,14.87,6.54,1,0,0,0,1
0610b486422d4921ae7d2bf64640c50b,5.50,15.00,11.20,0,1,0,0,0
38fe809add3b4fcf9315a9694bb96ff5,5.44,15.31,6.54,1,0,0,0,1
78afa995795e4d85b5d9ceeca43f5fef,7.50,14.94,10.00,0,1,0,0,0
a03223e636434f42ac4c3df47e8bac43,5.44,15.02,6.54,1,0,0,0,1
e2127556f4f64592b11af22de27a7932,6.80,15.25,7.00,0,0,1,0,0
8ec6ce2a7e7949b1bf142def7d0e0586,5.44,15.06,6.54,1,0,0,0,1
68617ca6246f4fbc85e91a2a49552598,5.44,15.07,6.54,1,0,0,0,1
389bc3fa690240e798340f5a15918d5c,6.50,15.18,5.30,0,0,1,0,0


# Transcript data set
Event log containing records for transactions, offers received, offers viewed, and offers completed

## Overview

In [12]:
transcript_df = load_dataframe(transcript_data_path)
display(transcript_df.head())

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


## Transformation

In [13]:
transcript_df = transcript_df.join(
    pd.DataFrame.from_records(transcript_df.pop('value')))
transcript_df.offer_id.update(transcript_df.pop('offer id'))

display(transcript_df)

,person,event,time,amount,offer_id,reward
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN
2,e2127556f4f64592b11af22de27a7932,offer received,0,NaN,2906b810c7d4411798c6938adc9daaa5,NaN
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,NaN,fafdcd668e3743c1bb461111dcafc2a4,NaN
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,NaN,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN
...,...,...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,714,1.59,NaN,NaN
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,714,9.53,NaN,NaN
306531,a00058cf10334a308c68e7631c529907,transaction,714,3.61,NaN,NaN
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,714,3.53,NaN,NaN


# Data cleaning

In [14]:
users_to_discard = np.append(
    transcript_df.query('amount >= 51').person.unique(),
    transcript_df.groupby('person').count().query('amount == 0').index.values)

profile_df = profile_df.query('id not in @users_to_discard')
transcript_df = transcript_df.query('person not in @users_to_discard')

# Classify offer sendings

In [15]:
## Split register according to event
offer_received_df = transcript_df.query('event == "offer received"')
offer_viewed_df = transcript_df.query('event == "offer viewed"')
offer_completed_df = transcript_df.query('event == "offer completed"')
transaction_df = transcript_df.query('event == "transaction"')

In [16]:
## Remove unnecessary columns
offer_received_df = offer_received_df.drop(columns=['amount', 'reward'])

## Create registers when customers did not receive offer
offer_sending_time = offer_received_df.time.unique()
new_index = pd.MultiIndex.from_product(
    (profile_df.index, offer_sending_time),
    names=['person', 'time'])

offer_received_df = offer_received_df.set_index(['person', 'time'])
offer_received_df = offer_received_df.sort_index()
offer_received_df = offer_received_df.reindex(new_index)
offer_received_df = offer_received_df.reset_index()

## Create column to indicate when the offer ends
offer_received_df = offer_received_df.join(portfolio_df.duration*24, on='offer_id')
offer_received_df = offer_received_df.assign(
    offer_ends_on=offer_received_df.time + offer_received_df.duration)

## Create column to indicate when the offer is informational
offer_received_df = offer_received_df.join(
    portfolio_df.informational, on='offer_id')

## Create column to indicate when the offer is viewed or completed
offer_received_df = offer_received_df.assign(viewed_on=np.nan, completed_on=np.nan)

## Create a column to hold the label for that offer sending
offer_received_df = offer_received_df.assign(label=np.nan)

In [17]:
%%time

next_sending_time = {0: 168, 168: 336, 336: 408,
                     408: 504, 504: 576, 576: 720}

def classify_offers(row):
    if row.offer_id is np.nan:
        row.offer_ends_on = next_sending_time[row.time]
        try:
            # In this case, there is no offer to be viewed or completed.
            # So we look for a simple transaction.
            row.completed_on = transaction_df.query(
                'person == @row.person ' \
                'and time >= @row.time ' \
                'and time <= @row.offer_ends_on').time.values[0]
        except:
            # If there is no transaction in this period
            row.label = 0
        else:
            row.label = 1
        finally:
            return row

    try:
        row.viewed_on = offer_viewed_df.query(
            'person == @row.person ' \
            'and offer_id == @row.offer_id ' \
            'and time >= @row.time ' \
            'and time <= @row.offer_ends_on').time.values[0]
    except:
        # Offer was not viewed
        row.label = 0
        return row

    
    if row.informational == 1:
        try:
            # In this case, there is no offer to be completed.
            # So we look for a simple transaction after offer viewed
            row.completed_on = transaction_df.query(
                'person == @row.person ' \
                'and time >= @row.viewed_on ' \
                'and time <= @row.offer_ends_on').time.values[0]
        except:
            # If there is no transaction in this period
            row.label = 0
        else:
            row.label = 1
        finally:
            return row


    try:
        # In the other cases, we need an offer completion
        row.completed_on = offer_completed_df.query(
            'person == @row.person ' \
            'and offer_id == @row.offer_id ' \
            'and time >= @row.viewed_on ' \
            'and time <= @row.offer_ends_on').time.values[0]
    except:
        # If the offer was not completed
        row.label = 0
    else:
        row.label = 1
    finally:
        return row


offer_received_df = offer_received_df.apply(classify_offers, axis=1)

## Remove auxiliary columns
offer_received_df.drop(
    inplace=True,
    columns=['duration', 'offer_ends_on', 'informational',
             'viewed_on', 'completed_on'])

CPU times: user 20min 35s, sys: 240 ms, total: 20min 36s
Wall time: 20min 36s


# Create Features

### Join datasets

In [18]:
## Set index as person and time
offer_received_df = offer_received_df.set_index(['person', 'time'])
offer_received_df = offer_received_df.sort_index()

## Join offer data with portfolio and profile data
offer_received_df = offer_received_df.join(portfolio_df, on='offer_id')
offer_received_df = offer_received_df.join(profile_df, on='person')

## Fill NA
offer_received_df.fillna(value=0, inplace=True)

display(offer_received_df)

event  \
person                           time                   
0009655768c64bdeb2e877511632db8f 0                  0   
                                 168   offer received   
                                 336   offer received   
                                 408   offer received   
                                 504   offer received   
...                                               ...   
ffff82501cea40309d5fdd7edcca4a07 168   offer received   
                                 336   offer received   
                                 408   offer received   
                                 504   offer received   
                                 576   offer received   

                                                               offer_id  \
person                           time                                     
0009655768c64bdeb2e877511632db8f 0                                    0   
                                 168   5a8bc65990b245e5a138643cd4eb9837   
                                 336   3f207df678b143eea3cee63160fa8bed   
                                 408   f19421c1d4aa40978ebb69ca19b0e20d   
                                 504   fafdcd668e3743c1bb461111dcafc2a4   
...                                                                 ...   
ffff82501cea40309d5fdd7edcca4a07 168   0b1e1539f2cc45b7b9fa7c272da2e1d7   
                                 336   2906b810c7d4411798c6938adc9daaa5   
                                 408   2906b810c7d4411798c6938adc9daaa5   
                                 504   9b98b8c7a33c4b65b9aebfe6a799e6d9   
                                 576   2906b810c7d4411798c6938adc9daaa5   

                                       label  reward  difficulty  duration  \
person                           time                                        
0009655768c64bdeb2e877511632db8f 0         0     0.0         0.0       0.0   
                                 168       1     0.0         0.0       3.0   
                                 336       1     0.0         0.0       4.0   
                                 408       0     5.0         5.0       5.0   
                                 504       0     2.0        10.0      10.0   
...                                      ...     ...         ...       ...   
ffff82501cea40309d5fdd7edcca4a07 168       1     5.0        20.0      10.0   
                                 336       1     2.0        10.0       7.0   
                                 408       1     2.0        10.0       7.0   
                                 504       0     5.0         5.0       7.0   
                                 576       0     2.0        10.0       7.0   

                                       bogo  discount  informational  mobile  \
person                           time                                          
0009655768c64bdeb2e877511632db8f 0      0.0       0.0            0.0     0.0   
                                 168    0.0       0.0            1.0     1.0   
                                 336    0.0       0.0            1.0     1.0   
                                 408    1.0       0.0            0.0     1.0   
                                 504    0.0       1.0            0.0     1.0   
...                                     ...       ...            ...     ...   
ffff82501cea40309d5fdd7edcca4a07 168    0.0       1.0            0.0     0.0   
                                 336    0.0       1.0            0.0     1.0   
                                 408    0.0       1.0            0.0     1.0   
                                 504    1.0       0.0            0.0     1.0   
                                 576    0.0       1.0            0.0     1.0   

                                       social  web  age  became_member_on  \
person                           time                                       
0009655768c64bdeb2e877511632db8f 0        0.0  0.0  3.3             14.93   
                                 168      1

### Plot the correlation matrix

In [19]:
offer_received_df.corr().abs().round(2)

,label,reward,difficulty,duration,bogo,discount,informational,mobile,social,web,age,became_member_on,income,missing_data,F,M,O,U
label,1.00,0.14,0.17,0.15,0.10,0.03,0.04,0.08,0.06,0.14,0.03,0.13,0.05,0.13,0.08,0.00,0.02,0.13
reward,0.14,1.00,0.63,0.51,0.82,0.02,0.38,0.40,0.48,0.30,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00
difficulty,0.17,0.63,1.00,0.85,0.20,0.67,0.41,0.14,0.19,0.55,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00
duration,0.15,0.51,0.85,1.00,0.22,0.70,0.17,0.53,0.35,0.72,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00
bogo,0.10,0.82,0.20,0.22,1.00,0.43,0.27,0.46,0.39,0.20,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
discount,0.03,0.02,0.67,0.70,0.43,1.00,0.27,0.11,0.07,0.54,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
informational,0.04,0.38,0.41,0.17,0.27,0.27,1.00,0.29,0.04,0.08,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
mobile,0.08,0.40,0.14,0.53,0.46,0.11,0.29,1.00,0.63,0.52,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00
social,0.06,0.48,0.19,0.35,0.39,0.07,0.04,0.63,1.00,0.12,0.00,0.01,0.00,0.00,0.00,0.00,0.01,0.00
web,0.14,0.30,0.55,0.72,0.20,0.54,0.08,0.52,0.12,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00


Here, we see features not correlated with each other, except for the gender *unknown* and *missing_data* in profile dataset. Thus, *missing_data* will be employed whereas *unknown* not.

### Organize customer data

In [20]:
%%time

## Certify that dataframe is ordered
offer_received_df = offer_received_df.sort_index()

## Choose features to feed the networks
features_cols = [
    'reward', 'difficulty', 'duration',                 # offer characteristics
    'bogo', 'discount', 'informational',                # offer type
    'mobile', 'social', 'web',                          # channels
    'age','became_member_on', 'income', 'missing_data', # customer data
    'F', 'M', 'O'                                       # customer gender
]

## Create feature and target lists where each position
# holds the data related to one customer
features = []
targets = []
for index, user_data in offer_received_df.groupby('person'):
    targets.append(user_data['label'].values)
    features.append(user_data[features_cols].values)

CPU times: user 15.9 s, sys: 32 ms, total: 15.9 s
Wall time: 15.5 s


### Create the data loaders

In [21]:
## Convert features and targets to tensors
features = torch.as_tensor(features, dtype=torch.float)
targets = torch.as_tensor(targets, dtype=torch.long)


## Split data into three random datasets 

# Generate randomic indices
len_dataset = len(features)
random_idx = np.random.choice(len_dataset, len_dataset, replace=False)

# Use the proportions: train: 80%, valid: 10%, test: 10%
train_idx = random_idx[:int(len_dataset*0.8)]
valid_idx = random_idx[int(len_dataset*0.8):-int(len_dataset*0.1)]
test_idx = random_idx[-int(len_dataset*0.1):]

# Create datasets
train_dataset = TensorDataset(features[train_idx], targets[train_idx])
valid_dataset = TensorDataset(features[valid_idx], targets[valid_idx])
test_dataset = TensorDataset(features[test_idx], targets[test_idx])

# Create dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

### Save the dataloaders into a zip file

In [22]:
torch.save((train_dataloader, valid_dataloader, test_dataloader), 'dataloaders.pt')
!zip dataloaders.zip dataloaders.pt
!rm dataloaders.pt

  adding: dataloaders.pt (deflated 93%)
